In [ ]:
from PyPDF2 import PdfReader
import pandas as pd
import re
import numpy as np

pdf_path = "sem10.pdf"
page = PdfReader(pdf_path).pages[56]

def preprocess(line):
    return re.split(r'\s{2,}', line.replace(',',''))

lines = map(preprocess, page.extract_text().split('\n'))

entidades = [
    "Aguascalientes", "Baja California", "Baja California Sur", "Campeche", 
    "Coahuila", "Colima", "Chiapas", "Chihuahua", "Distrito Federal", 
    "Durango", "Guanajuato", "Guerrero", "Hidalgo", "Jalisco", "México", 
    "Michoacán", "Morelos", "Nayarit", "Nuevo León", "Oaxaca", "Puebla", 
    "Querétaro", "Quintana Roo", "San Luis Potosí", "Sinaloa", "Sonora", 
    "Tabasco", "Tamaulipas", "Tlaxcala", "Veracruz", "Yucatán", "Zacatecas", "TOTAL"
]

def filter_by_entity(line):
    if ' ' not in line[0] and line[0] in entidades:
        return True
    
    return " ".join(line[0].split(' ')[:-1]) in entidades

filtered_lines = list(filter(filter_by_entity, lines))
filtered_lines        

TypeError: 'type' object is not subscriptable

In [ ]:
def parse_first_item(item):
    """
    Separa el nombre del estado (parte de texto) de cualquier número que
    esté pegado al final.
    """
    # Reemplaza múltiples espacios por uno solo (por seguridad)
    item = re.sub(r'\s+', ' ', item).strip()
    
    # Busca una parte sin dígitos seguida opcionalmente de algo numérico
    match = re.match(r'([^\d]+)(.*)', item)
    if match:
        state_part = match.group(1).strip()  # texto (nombre estado)
        number_part = match.group(2).strip() # lo que quede (puede ser dígitos u otro)
    else:
        # Si no macha, consideramos todo como nombre
        state_part = item
        number_part = ""
    return state_part, number_part

def split_and_clean(items):
    """
    - Si el string contiene guiones, lo parte por espacios ('25 -' -> ['25','-']).
    - Si NO contiene guiones ('1 047'), elimina espacios internos para leerlo como un solo número ('1047').
    - Convierte dígitos a int, guiones a NaN.
    """
    output = []
    for piece in items:
        # Caso 1: Si hay un guion, partimos por espacio
        if '-' in piece:
            tokens = piece.split()
            for t in tokens:
                t = t.strip().replace(' ', '')  # elimina espacios internos
                if t.isdigit():
                    output.append(int(t))
                elif t == '-':
                    output.append(np.nan)
                else:
                    output.append(np.nan)
        else:
            # Caso 2: No hay guion => eliminar todos los espacios internos y tratarlo como un único valor
            piece_no_spaces = piece.replace(' ', '')
            if piece_no_spaces.isdigit():
                output.append(int(piece_no_spaces))
            else:
                # Si no es dígito limpio, lo convertimos a NaN
                output.append(np.nan)
    return output

cleaned_data = []
for row in filtered_lines:
    # 1) Separar el primer item (nombre estado + posible número)
    state, possible_num = parse_first_item(row[0])

    # 2) Construir lista con "possible_num" (si lo hay) y el resto de la fila
    #    a partir del segundo elemento.
    rest = []
    if possible_num:
        rest.append(possible_num)
    rest.extend(row[1:])  # Agregar el resto

    # 3) Dividir cada sub-elemento y limpiar
    numeric_values = split_and_clean(rest)

    # -- En este paso numeric_values puede tener más o menos de 9 elementos. --
    # Si quieres forzar a que cada fila tenga EXACTAMENTE 9 valores:
    if len(numeric_values) < 9:
        # Rellenar con NaN si faltan
        numeric_values += [np.nan]*(9 - len(numeric_values))
    elif len(numeric_values) > 9:
        # Recortar si sobran
        numeric_values = numeric_values[:9]

    # 4) Agregar la fila completa: [Estado] + 9 valores
    cleaned_data.append([state] + numeric_values)

# Definir columnas
columns = [
    'Entidad_Federativa',
    'Depresion_Sem','Depresion_M','Depresion_F',
    'Parkinson_Sem','Parkinson_M','Parkinson_F',
    'Alzheimer_Sem','Alzheimer_M','Alzheimer_F'
]

# Crear DataFrame
df = pd.DataFrame(cleaned_data, columns=columns)

# FORMATEAR NÚMEROS MAYORES DE 999
# (aunque con estos datos quizá no sea muy común)
def formato_miles(x):
    if pd.isna(x):
        return np.nan
    # Si es entero y > 999, poner formato "1,234"
    if isinstance(x, int) and x > 999:
        return f"{x:,}".replace(",", " ")
    return x

df.iloc[:, 1:] = df.iloc[:, 1:].applymap(formato_miles)

df

,Entidad_Federativa,Depresion_Sem,Depresion_M,Depresion_F,Parkinson_Sem,Parkinson_M,Parkinson_F,Alzheimer_Sem,Alzheimer_M,Alzheimer_F
0,Aguascalientes,2,7,17,NaN,NaN,2.0,NaN,NaN,NaN
1,Baja California,80,80,320,5.0,24.0,27.0,2.0,5.0,16.0
2,Baja California Sur,10,13,45,NaN,4.0,4.0,1.0,NaN,1.0
3,Campeche,11,24,87,3.0,4.0,6.0,1.0,4.0,4.0
4,Coahuila,33,58,189,6.0,29.0,26.0,1.0,3.0,7.0
5,Colima,9,30,96,3.0,7.0,14.0,NaN,NaN,2.0
6,Chiapas,17,19,110,5.0,13.0,7.0,1.0,4.0,5.0
7,Chihuahua,79,80,357,8.0,28.0,27.0,3.0,2.0,13.0
8,Distrito Federal,273,468,1 234,5.0,27.0,73.0,1.0,6.0,5.0
9,Durango,55,45,115,NaN,2.0,2.0,NaN,2.0,3.0
